<a href="https://colab.research.google.com/github/url-github/Data-Engineering/blob/main/2_Spotify_2_TRANSFORM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.youtube.com/playlist?list=PLNkCniHtd0PNM4NZ5etgYMw4ojid0Aa6i

Transform - chodzi o weryfikację czy dane są dobre np. plik JSON nie jest pusty, albo nie ma tam duplikatów.

Weryfikacja (Integralność danych) w poniższym kodzie sprowadza się do funkcji `check_if_valid_data`

## III wersja

In [ ]:
import sqlalchemy
import pandas as pd 
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3


DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"
USER_ID = "31yahlyxg6b7b27ffx4qrguic5e4" # your Spotify username 
TOKEN = "BQCLrthKISKuBifWf8YK5-mav1P6zXOsiUmUYNwinEmE2KBBsjtjjfCTR4v7DjpG7MprNVw2axhEzf_y-L0YH4A6cRqWO9Tjq0rHgtLbCYQ8hfM6DNaxyaEDK20YQlFeotjiEJFXtDtTD3cnTeN7S8P6dLMmfStLXwItggy0hfoIP154RrYGXhWWR5-sHcF3OGNemAyW" # your Spotify API token

# Generate your token here:  https://developer.spotify.com/console/get-recently-played/
# Note: You need a Spotify account (can be easily created for free)

def check_if_valid_data(df: pd.DataFrame) -> bool:
    # Sprawdź, czy dataframe jest pusty
    if df.empty:
        print("No songs downloaded. Finishing execution") # Nie pobrano żadnych utworów. 
        return False 

    # Sprawdzenie klucza podstawowego (Primary Key Check)
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("Primary Key check is violated") # Kontrola klucza podstawowego jest naruszona

    # Sprawdź, czy nie ma wartości null
    if df.isnull().values.any():
        raise Exception("Null values found") # Znaleziono null

    # Sprawdź, czy wszystkie znaczniki czasu pochodzą z wczorajszej daty
    yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
    yesterday = yesterday.replace(hour=0, minute=0, second=0, microsecond=0)

    timestamps = df["timestamp"].tolist()
    for timestamp in timestamps:
        if datetime.datetime.strptime(timestamp, '%Y-%m-%d') != yesterday:
            raise Exception("At least one of the returned songs does not have a yesterday's timestamp")
            # Co najmniej jeden ze zwróconych utworów nie ma wczorajszego znacznika czasu

    return True

if __name__ == "__main__":

    # Wyodrębnij część procesu ETL
 
    headers = {
        "Accept" : "application/json",
        "Content-Type" : "application/json",
        "Authorization" : f'Bearer {TOKEN}'
    }
    
    # Konwertuj czas na uniksowy znacznik czasu w milisekundach     
    today = datetime.datetime.now()
    yesterday = today - datetime.timedelta(days=1)
    yesterday_unix_timestamp = int(yesterday.timestamp()) * 1000

    # Pobierz wszystkie utwory, których słuchałeś „po wczoraj”, czyli w ciągu ostatnich 24 godzin    
    r = requests.get("https://api.spotify.com/v1/me/player/recently-played?after={time}".format(time=yesterday_unix_timestamp), headers = headers)

    data = r.json()

    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []

    # Wyodrębnianie tylko odpowiednich danych z obiektu json     
    for song in data["items"]:
        song_names.append(song["track"]["name"])
        artist_names.append(song["track"]["album"]["artists"][0]["name"])
        played_at_list.append(song["played_at"])
        timestamps.append(song["played_at"][0:10])

    # Przygotuj słownik, aby przekształcić go w data frame poniżej       
    song_dict = {
        "song_name" : song_names,
        "artist_name": artist_names,
        "played_at" : played_at_list,
        "timestamp" : timestamps
    }

    song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])

    print(song_df)

                                            song_name  \
0          Comfortably Numb - 2011 Remastered Version   
1      Wots ... Uh The Deal - 2011 Remastered Version   
2                         Cluster One - 2011 Remaster   
3                      Time - 2011 Remastered Version   
4              Interstellar Overdrive - 2011 Remaster   
5               Us And Them - 2011 Remastered Version   
6             The Final Cut - 2011 Remastered Version   
7                    Sorrow - 2011 Remastered Version   
8                                  Wish You Were Here   
9   Shine On You Crazy Diamond, Pts. 1-5 - 2011 Re...   
10                   Astronomy Domine - 2011 Remaster   
11                 Fearless - 2011 Remastered Version   
12             Arnold Layne - 1996 Remastered Version   
13             Brain Damage - 2011 Remastered Version   
14                        Poles Apart - 2011 Remaster   
15     Breathe (In The Air) - 2011 Remastered Version   
16                             